# Input and output

In this unit we will discuss input and output,
or I/O for short. In doing so, we will focus exclusively on I/O
routines used to load and store data from files that are relevant
for numerical computation and data analysis.

## I/O with NumPy

We already encountered the most basic, and probably most frequently used
NumPy I/O routine, `np.loadtxt()`. The most important I/O functions to
process text data are:
-   `loadtxt()`: load data from a text file.
-   `genfromtxt()`: load data from a text file and handle missing
    data.
-   `savetxt()`: save a NumPy array to a text file.

There are a few other I/O functions in NumPy, for example to
write arrays as raw binary data.
These are listed in the [official documentation](https://numpy.org/doc/stable/reference/routines.io.html).

We frequently use files that store data
as character-separated values (CSV) in pure text format since virtually
and application supports this data format.

Imagine we have the following tabular data from [FRED](https://fred.stlouisfed.org/)
which we already used in the first unit, where the first two rows
look as follows:

| Year |  GDP   |  CPI | UNRATE |
| ---- | ------ | ---- | ------ |
| 1948 | 2118.5 | 24.0 | 3.8    |
| 1949 | 2106.6 | 23.8 | 6.0    |

To load a CSV file as a NumPy array, we use `loadtxt()`:

In [3]:
import numpy as np

# load CSV
data = np.loadtxt('../data/FRED.csv', skiprows=1, delimiter=',')
data[:2]        # Display first two rows

array([[1948. , 2118.5,   24. ,    3.8],
       [1949. , 2106.6,   23.8,    6. ]])

The default arguments will in many cases be appropriate to load
whatever CSV file you might have. However, you'll occassionally
want to specify the following arguments to override the defaults:

-   `delimiter`: Character used to separate individual fields (default: space)
-   `skiprows=n`: Skip the first `n` rows. For example, if the CSV file
    contains a header with variable names, `skiprows=1` needs to be
    specified.
-   `dtype`: Enforce a particular data type for the resulting array.
-   `encoding`: Set the character encoding of the input data. This
    will hardly ever be needed, but can be required to import data
    with non-latin characters that are not encoded using Unicode.

NumPy implements an additional function to load text data,
`np.genfromtxt()`. This routine is more flexible: for example, it can handle
missing data.

Finally, to save a NumPy array to a CSV file there is a logical counterpart
to `np.loadtxt()` which is called `np.savetxt()`.

In [7]:
import numpy as np
import os.path
import tempfile

# Generate some random data on [0,1)
data = np.random.default_rng(123).random(size=(10, 5))

# create temporary directory
d = tempfile.TemporaryDirectory()
# CSV file name
filename = os.path.join(d.name, 'data.csv')

# Write NumPy array to CSV file
np.savetxt(filename, data, delimiter=';', fmt='%8.5f')

The above code creates a $10 \times 5$ matrix of random floats
and stores these in the file `data.csv` using 5 significant digits.
The destination file is located in a temporary directory which
will be different on every system this code is run on
We use the `tempfile` module to create this writable temporary
directory.


***
## I/O with Pandas

Pandas's I/O routines are more powerful than those implemented in NumPy,
and they support reading numerous file formats. Since pandas supports heterogeneous data,
it will additionally import files with mixed data (eg. strings and floats),
whereas NumPy creates a somewhat useless array of `object` type.

The most important routines are:

-   `read_csv()`, `to_csv()`: Read or write CSV text files
-   `read_fwf()`: Read data with fixed field with, ie. text data
    that does not use delimiters to separate fields.
-   `read_excel()`, `to_excel()`: Read or write Excel spreadsheets
-   `read_stata()`, `to_stata()`: Rear of write Stata `.dta` files.

For a complete list of I/O routines, see the [official documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).

To illustrate, we repeat the above example using pandas's
`read_csv()`. Since this file contains only floating-point
data, the result is very similar to reading in a NumPy array.

In [11]:
import pandas as pd

filename = '../data/FRED.csv'
df = pd.read_csv(filename, sep=',')
df.head(2)          # Display the first 2 rows of data

,Year,GDP,CPI,UNRATE
0,1948,2118.5,24.0,3.8
1,1949,2106.6,23.8,6.0
2,1950,2289.5,24.1,5.2
3,1951,2473.8,26.0,3.3
4,1952,2574.9,26.6,3.0


The difference between NumPy and pandas become obvious when we
try to load our university data from an earlier unit:

In [12]:
import numpy as np

filename = '../data/universities.csv'
# Try to load CSV data that contains strings
# This will result in an error!
data = np.loadtxt(filename, delimiter=';')

ValueError: could not convert string to float: '"Institution"'

On the other hand, this mixed data can be loaded with pandas
without issues:

In [13]:
import pandas as pd

filename = '../data/universities.csv'
df = pd.read_csv(filename, sep=';')
df.head(3)      # show first 3 rows

,Institution,Country,Founded,Students,Budget,Rank
0,University of Glasgow,Scotland,1451,30805,626.5,92
1,University of Edinburgh,Scotland,1583,34275,1102.0,30
2,University of St Andrews,Scotland,1413,8984,251.2,201


Unlike NumPy, pandas can also process other popular data formats
such as MS Excel files (or OpenDocument spreadsheets):

In [1]:
import pandas as pd

# Excel file containing university data
filename = '../data/../data/universities.xlsx'

df = pd.read_excel(filename, sheet_name='universities')
df.head(3)

,Institution,Country,Founded,Students,Budget,Rank
0,University of Glasgow,Scotland,1451,30805,626.5,92
1,University of Edinburgh,Scotland,1583,34275,1102.0,30
2,University of St Andrews,Scotland,1413,8984,251.2,201


The routine `read_excel()` takes the argument `sheet_name` to
specify the sheet that should be read.

Note the Python package `xlrd` needs to be installed in order
to read files from Excel 2003 and above.

Finally, we often encounter text files with fixed field widths,
since this is a commonly used format in older applications
(for example, fixed-width files are easy to create in Fortran).
To read such files, the width (ie. the number of characters)
has to be explicitly specified:

In [2]:
import pandas as pd

# File name of FRED data, stored as fixed-width text
filename = '../data/FRED-fixed.csv'

# field widths are passed as list to read_fwf()
df = pd.read_fwf(filename, widths=[5, 7, 5, 8])
df.head(3)


,Year,GDP,CPI,UNRATE
0,1948,2118.5,24.0,3.8
1,1949,2106.6,23.8,6.0
2,1950,2289.5,24.1,5.2


***
## Pickling

A wholly different approach to data I/O is taken by Python's
built-in `pickle` module (see [official documentation](https://docs.python.org/3/library/pickle.html)).
Almost any Python object can be
dumped into a binary file and read back using `pickle.dump()`
and `pickle.load()`.

The big advantage over other methods
is that hierarchies of objets are automatically supported.
For example, we can pickle a list containing a `tuple`, a string and a NumPy array:

In [10]:
import numpy as np
import pickle
import tempfile
import os.path

# Generate some random data on [0,1)
arr = np.arange(10).reshape((2, -1))
tpl = (1, 2, 3)
text = 'Pickle is very powerful!'

# data: several nested containers and strings
data = [tpl, text, arr]

# create temporary directory
d = tempfile.TemporaryDirectory()
# Binary destination file
filename = os.path.join(d.name, 'data.bin')

with open(filename, 'wb') as f:
    pickle.dump(data, f)

We can then read back the data as follows:

In [12]:
# load pickle data from above
with open(filename, 'rb') as f:
    data = pickle.load(f)

# expand list into its components
tpl, text, arr = data
arr         # prints previously generated array

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

The above example introduces a few concepts we have not countered so far:

1.  The built-in function `open()` is used to open files for
    reading or writing. The second argument indicates whether
    a file should be read-only, `r`, or writable, `w`.
    The `b` sets the file mode to *binary*, ie its contents are not
    human-readable text.
2.  We usually access files using a so-called context manager.
    A context manager is created using the `with` statement.

    A big advantage of using a context manager is that the
    file resource made available as `f` in the block following
    `with` is automatically cleaned up as soon as the block exits.

So why not always use `pickle` to load and store data?

1.  Pickling is Python-specific and no other application can process
    pickled data.
2.  The pickle protocol can change in a newer version of Python,
    and you might not be able to read back your old pickled objects.
3.  Even worse, because projects such as NumPy and Pandas implement
    their own pickling routines, you might not even be able
    to unpickle old DataFrames when you upgrade to a newer Pandas version!
4.  `pickle` is not secure: It is possible to construct binary
    data that will execute arbitrary code when unpickling, so you
    don't want to unpickle data from untrusted sources.
5.  Some object cannot be pickled. For example, this applies to
    any classes defined using Numba or Cython.
    As a novice programmer you will not define such data types yourself,
    but might unknowingly encounter them if they are part of a library
    you use.


`pickle` is great for internal use when you do not need to exchange
data with others, and have complete control over your computing environment
(ie. you can enforce a specific version of Python and the libraries you are
using). For anything else, you should avoid it.